In [21]:
from email.utils import format_datetime
from itertools import count
from sys import displayhook
import xml.etree.ElementTree as ET
import pandas as pd
import glob # Библиотека для получения списка файлов
import os
import stat
from os import listdir # Библиотека для получения списка файлов
from os.path import isfile, join
from pathlib import Path


# text processing libraries
import re
import string
# import nltk
# from nltk.corpus import stopwords
from datetime import date, datetime, time
#import plotly.express as px

# Библиотеки для визуализации
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import plotly.graph_objects as go

import librosa
from librosa import get_duration  #  pip install librosa ## библ для рассчета продолжительности записи


In [12]:
# Обозначаем путь нахождения XML файлов 

#path = '/Users/ekaterina/github_repos/parsing_XML/AIFD_XML_full/20220426_gran_entire_day/xml'
# path = '/Users/ekaterina/data4repos/AIFD_XML_full/20220426_gran_entire_day/'
#/Users/ekaterina/data4repos/AIFD_XML_full/20220426_gran_entire_day

path = '/media/victor/20220427_gran_entire_day/'


"""
# Еще можно так получить список файлов, но в данном случае нельзя указать расширение файла (например xml)
list_of_files=os.listdir(path)
print('Список всех файлов: ',len(list_of_files))
# print('Список всех файлов: ',list_of_files) список всех файлов - работает
"""
# Получаем список файлов
list_of_files = glob.glob(path+ '/*.xml')
df_flist = pd.DataFrame(list_of_files)
# print('\nСписок всех файлов:\n',list_of_files)

# Визуализация таблицы со списком файлов
"""
fig = go.Figure(data=[go.Table(
    #header=dict(values=list(df_flist.columns),
    #            fill_color='paleturquoise',
    #            align='right'),
    cells=dict(values=[df_flist],
               fill_color='lavender',
               align='right'),)
])
fig.update_layout(width=1000, height=500)
fig.show()
"""
df_flist

,0
0,/media/victor/20220427_gran_entire_day/node_7_...
1,/media/victor/20220427_gran_entire_day/node_7_...
2,/media/victor/20220427_gran_entire_day/node_7_...
3,/media/victor/20220427_gran_entire_day/node_7_...
4,/media/victor/20220427_gran_entire_day/node_8_...
...,...
39995,/media/victor/20220427_gran_entire_day/node_8_...
39996,/media/victor/20220427_gran_entire_day/node_8_...
39997,/media/victor/20220427_gran_entire_day/node_8_...
39998,/media/victor/20220427_gran_entire_day/node_7_...


In [13]:
# Чтение первого файла 

file_path_file1 = os.path.join(path, list_of_files[0])
tree = ET.parse(file_path_file1)
root = tree.getroot()

print('Чтение первого файла:')
print(root.tag, root.attrib)

print('\nПуть к 1-му файлу:')
print(file_path_file1)
print()

Чтение первого файла:
analysisTask {'cc-task': '7_3_7_1645875244_3624108'}

Путь к 1-му файлу:
/media/victor/20220427_gran_entire_day/node_7_domain_3_nauss_7_1645875244_3624108.xml



In [14]:
# просмотр дочерних элементов и атрибутов
for child in root:     
    print('Дочерние элементы и атрибуты:', child.tag, child.attrib)

# просмотр всех атрибутов и элементов документа
print('\nПросмотр всех атрибутов и элементов документа:\n', ET.tostring(root, encoding='utf8').decode('utf8'))




Дочерние элементы и атрибуты: timestamp {}
Дочерние элементы и атрибуты: filename {}
Дочерние элементы и атрибуты: channel {'type': 'a', 'ccAgentID': 'lobanova_ea_ksm', 'ccDeviceID': 'lobanova_ea_ksm'}
Дочерние элементы и атрибуты: channel {'type': 'c', 'value': '+7(985)137-98-48', 'caller': 'true', 'disconnect': 'true'}

Просмотр всех атрибутов и элементов документа:
 <?xml version='1.0' encoding='utf8'?>
<analysisTask cc-task="7_3_7_1645875244_3624108"><timestamp>2022-04-28 15:33:07.000 MSK</timestamp><filename>node_7_domain_3_nauss_7_1645875244_3624108.wav</filename><channel type="a" ccAgentID="lobanova_ea_ksm" ccDeviceID="lobanova_ea_ksm" /><channel type="c" value="+7(985)137-98-48" caller="true" disconnect="true" /></analysisTask>


In [15]:
# ФОРМИРОВАНИЕ РАБОЧЕГО ДАТА-ФРЕЙМА

# инициализируем два пустых фрейма данных, которые будут заполнены указанными выше элементами. 
# Мы включим type,ccAgentID, ccDeviceID, timestamp, caller
df_1 = pd.DataFrame()
df = pd.DataFrame()
i=0
list_keywords=[]

for file in list_of_files:
    # file_path=path+file  - в оригинале(у нас path прописан path = '/Users/ekaterina/github_repos/parsing_XML/xml' поэтому происходит дублирование - path+path+file - выдает ошибку. В связи с этим надо оставить file_path=file)
    # print("Директория:\n", path)
    # print('Файл:\n', file)
    # print('Путь к файлу:\n'+file_path)
    # print('~'*40)
    tree = ET.parse(file)
    #tree = ET.parse(path+file)
    root = tree.getroot()
# Формируем колонки:
    trial = {}

    # trial['type'] = root.find('channel').text         # не требуется для анализа
    trial['ccAgentID'] = root.find('channel').text 
    # trial['ccDeviceID'] = root.find('channel').text   # не требуется для анализа
    trial['filename'] = root.find('filename').text

# Переводим строковое значение времени в datestamp
    date_string = root.find('timestamp').text
    format_datetime = pd.to_datetime(date_string)
    format_date = pd.to_datetime(date_string).date()
    format_time = pd.to_datetime(date_string).time()
    # print(format_time) # проверка 

    """
    if root.find('channel') != None:
        trial['type'] = tree.find('channel').get('type')
    else:
        trial['type'] = ''
    """
        
    if root.find('channel') != None:
        trial['ccAgentID'] = tree.find('channel').get('ccAgentID')
    else:
        trial['ccAgentID'] = ''
    '''
    if root.find('channel') != None:
        trial['ccDeviceID'] = tree.find('channel').get('ccDeviceID')
    else:
        trial['ccDeviceID'] = ''
    '''
    if root.find('timestamp') != None:
        trial['datetime'] = format_datetime    
    else:
        trial['datetime'] = ''

    if root.find('timestamp') != None:
        trial['date'] = format_date   
    else:
        trial['date'] = ''

    if root.find('timestamp') != None:
        trial['time'] = format_time   
    else:
        trial['time'] = ''
    """
    if root.find('filename') != None:
        trial['filename'] = tree.find('channel').get('filename')
    else:
        trial['filename'] = ''
    """
    df = pd.DataFrame(trial,index=[i]) # i-ая строка из df...
    i=i+1

    df_1 = pd.concat([df_1, df])       # ...присоединяется к df_1 


In [16]:
df_1

,ccAgentID,filename,datetime,date,time
0,lobanova_ea_ksm,node_7_domain_3_nauss_7_1645875244_3624108.wav,2022-04-28 15:33:07+03:00,2022-04-28,15:33:07
1,kolenchukoz,node_7_domain_6_nauss_7_1645875244_3958744.wav,2022-05-06 20:00:50+03:00,2022-05-06,20:00:50
2,perminova_li,node_7_domain_1_nauss_7_1645875244_4200459.wav,2022-05-13 10:35:19+03:00,2022-05-13,10:35:19
3,kostinaoz,node_7_domain_5_nauss_7_1645875244_3776968.wav,2022-05-02 19:06:50+03:00,2022-05-02,19:06:50
4,kostinaoz,node_8_domain_3_nauss_8_1645876859_4411762.wav,2022-05-06 20:37:43+03:00,2022-05-06,20:37:43
...,...,...,...,...,...
39995,abzalov,node_8_domain_6_nauss_8_1645876859_4469450.wav,2022-05-09 08:25:53+03:00,2022-05-09,08:25:53
39996,gontarenkooz,node_8_domain_4_nauss_8_1645876859_4154704.wav,2022-04-30 14:46:14+03:00,2022-04-30,14:46:14
39997,harsheva,node_8_domain_1_nauss_8_1645876859_4021438.wav,2022-04-28 10:59:51+03:00,2022-04-28,10:59:51
39998,kulikova_eu_ksm,node_7_domain_4_nauss_7_1645875244_3672169.wav,2022-04-29 12:09:48+03:00,2022-04-29,12:09:48


In [17]:
df_2 = df_1.assign(Duration = 0.0)

In [18]:
df_2

,ccAgentID,filename,datetime,date,time,Duration
0,lobanova_ea_ksm,node_7_domain_3_nauss_7_1645875244_3624108.wav,2022-04-28 15:33:07+03:00,2022-04-28,15:33:07,0.0
1,kolenchukoz,node_7_domain_6_nauss_7_1645875244_3958744.wav,2022-05-06 20:00:50+03:00,2022-05-06,20:00:50,0.0
2,perminova_li,node_7_domain_1_nauss_7_1645875244_4200459.wav,2022-05-13 10:35:19+03:00,2022-05-13,10:35:19,0.0
3,kostinaoz,node_7_domain_5_nauss_7_1645875244_3776968.wav,2022-05-02 19:06:50+03:00,2022-05-02,19:06:50,0.0
4,kostinaoz,node_8_domain_3_nauss_8_1645876859_4411762.wav,2022-05-06 20:37:43+03:00,2022-05-06,20:37:43,0.0
...,...,...,...,...,...,...
39995,abzalov,node_8_domain_6_nauss_8_1645876859_4469450.wav,2022-05-09 08:25:53+03:00,2022-05-09,08:25:53,0.0
39996,gontarenkooz,node_8_domain_4_nauss_8_1645876859_4154704.wav,2022-04-30 14:46:14+03:00,2022-04-30,14:46:14,0.0
39997,harsheva,node_8_domain_1_nauss_8_1645876859_4021438.wav,2022-04-28 10:59:51+03:00,2022-04-28,10:59:51,0.0
39998,kulikova_eu_ksm,node_7_domain_4_nauss_7_1645875244_3672169.wav,2022-04-29 12:09:48+03:00,2022-04-29,12:09:48,0.0


In [19]:
%conda list librosa

# packages in environment at /home/victor/anaconda3/envs/etensor:
#
# Name                    Version                   Build  Channel
librosa                   0.10.0             pyhd8ed1ab_2    conda-forge

Note: you may need to restart the kernel to use updated packages.


## Call duration add

In [22]:
i = 0
for i in range (len(df_2)):
    wav_file = df_2.filename[i]
    df_2.Duration[i] = librosa.get_duration(path = path+wav_file)

/tmp/ipykernel_1206850/1494358778.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.Duration[i] = librosa.get_duration(path = path+wav_file)


In [23]:
df_2

,ccAgentID,filename,datetime,date,time,Duration
0,lobanova_ea_ksm,node_7_domain_3_nauss_7_1645875244_3624108.wav,2022-04-28 15:33:07+03:00,2022-04-28,15:33:07,98.062
1,kolenchukoz,node_7_domain_6_nauss_7_1645875244_3958744.wav,2022-05-06 20:00:50+03:00,2022-05-06,20:00:50,10.937
2,perminova_li,node_7_domain_1_nauss_7_1645875244_4200459.wav,2022-05-13 10:35:19+03:00,2022-05-13,10:35:19,57.094
3,kostinaoz,node_7_domain_5_nauss_7_1645875244_3776968.wav,2022-05-02 19:06:50+03:00,2022-05-02,19:06:50,10.363
4,kostinaoz,node_8_domain_3_nauss_8_1645876859_4411762.wav,2022-05-06 20:37:43+03:00,2022-05-06,20:37:43,151.826
...,...,...,...,...,...,...
39995,abzalov,node_8_domain_6_nauss_8_1645876859_4469450.wav,2022-05-09 08:25:53+03:00,2022-05-09,08:25:53,228.782
39996,gontarenkooz,node_8_domain_4_nauss_8_1645876859_4154704.wav,2022-04-30 14:46:14+03:00,2022-04-30,14:46:14,9.589
39997,harsheva,node_8_domain_1_nauss_8_1645876859_4021438.wav,2022-04-28 10:59:51+03:00,2022-04-28,10:59:51,17.878
39998,kulikova_eu_ksm,node_7_domain_4_nauss_7_1645875244_3672169.wav,2022-04-29 12:09:48+03:00,2022-04-29,12:09:48,72.798


In [24]:
# Function for transfer digit in Duration (seconds) to datetime_format

def fun(seconds):
    mn,sc = divmod(seconds,60)
    hr,mn = divmod(mn,60)
    return f"{hr}:{mn}:{sc}"


In [25]:
# Just for check 
str(datetime.timedelta(seconds=98.9))

AttributeError: type object 'datetime.datetime' has no attribute 'timedelta'

In [26]:
df_2.Duration.apply(fun)


0                    0.0:1.0:38.062
1                    0.0:0.0:10.937
2                    0.0:0.0:57.094
3                    0.0:0.0:10.363
4        0.0:2.0:31.825999999999993
                    ...            
39995     0.0:3.0:48.78200000000001
39996                 0.0:0.0:9.589
39997                0.0:0.0:17.878
39998    0.0:1.0:12.798000000000002
39999                0.0:0.0:58.957
Name: Duration, Length: 40000, dtype: object

In [27]:
df_2['Duration_sec'] = df_2['Duration'].apply(fun)
df_2

,ccAgentID,filename,datetime,date,time,Duration,Duration_sec
0,lobanova_ea_ksm,node_7_domain_3_nauss_7_1645875244_3624108.wav,2022-04-28 15:33:07+03:00,2022-04-28,15:33:07,98.062,0.0:1.0:38.062
1,kolenchukoz,node_7_domain_6_nauss_7_1645875244_3958744.wav,2022-05-06 20:00:50+03:00,2022-05-06,20:00:50,10.937,0.0:0.0:10.937
2,perminova_li,node_7_domain_1_nauss_7_1645875244_4200459.wav,2022-05-13 10:35:19+03:00,2022-05-13,10:35:19,57.094,0.0:0.0:57.094
3,kostinaoz,node_7_domain_5_nauss_7_1645875244_3776968.wav,2022-05-02 19:06:50+03:00,2022-05-02,19:06:50,10.363,0.0:0.0:10.363
4,kostinaoz,node_8_domain_3_nauss_8_1645876859_4411762.wav,2022-05-06 20:37:43+03:00,2022-05-06,20:37:43,151.826,0.0:2.0:31.825999999999993
...,...,...,...,...,...,...,...
39995,abzalov,node_8_domain_6_nauss_8_1645876859_4469450.wav,2022-05-09 08:25:53+03:00,2022-05-09,08:25:53,228.782,0.0:3.0:48.78200000000001
39996,gontarenkooz,node_8_domain_4_nauss_8_1645876859_4154704.wav,2022-04-30 14:46:14+03:00,2022-04-30,14:46:14,9.589,0.0:0.0:9.589
39997,harsheva,node_8_domain_1_nauss_8_1645876859_4021438.wav,2022-04-28 10:59:51+03:00,2022-04-28,10:59:51,17.878,0.0:0.0:17.878
39998,kulikova_eu_ksm,node_7_domain_4_nauss_7_1645875244_3672169.wav,2022-04-29 12:09:48+03:00,2022-04-29,12:09:48,72.798,0.0:1.0:12.798000000000002


In [28]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 39999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype                                
---  ------        --------------  -----                                
 0   ccAgentID     40000 non-null  object                               
 1   filename      40000 non-null  object                               
 2   datetime      40000 non-null  datetime64[ns, pytz.FixedOffset(180)]
 3   date          40000 non-null  object                               
 4   time          40000 non-null  object                               
 5   Duration      40000 non-null  float64                              
 6   Duration_sec  40000 non-null  object                               
dtypes: datetime64[ns, pytz.FixedOffset(180)](1), float64(1), object(5)
memory usage: 3.4+ MB


In [29]:
#  Add column 'End_time_conversation'
df_3 = df_2.assign(Raznica = 0.0)
df_3

,ccAgentID,filename,datetime,date,time,Duration,Duration_sec,Raznica
0,lobanova_ea_ksm,node_7_domain_3_nauss_7_1645875244_3624108.wav,2022-04-28 15:33:07+03:00,2022-04-28,15:33:07,98.062,0.0:1.0:38.062,0.0
1,kolenchukoz,node_7_domain_6_nauss_7_1645875244_3958744.wav,2022-05-06 20:00:50+03:00,2022-05-06,20:00:50,10.937,0.0:0.0:10.937,0.0
2,perminova_li,node_7_domain_1_nauss_7_1645875244_4200459.wav,2022-05-13 10:35:19+03:00,2022-05-13,10:35:19,57.094,0.0:0.0:57.094,0.0
3,kostinaoz,node_7_domain_5_nauss_7_1645875244_3776968.wav,2022-05-02 19:06:50+03:00,2022-05-02,19:06:50,10.363,0.0:0.0:10.363,0.0
4,kostinaoz,node_8_domain_3_nauss_8_1645876859_4411762.wav,2022-05-06 20:37:43+03:00,2022-05-06,20:37:43,151.826,0.0:2.0:31.825999999999993,0.0
...,...,...,...,...,...,...,...,...
39995,abzalov,node_8_domain_6_nauss_8_1645876859_4469450.wav,2022-05-09 08:25:53+03:00,2022-05-09,08:25:53,228.782,0.0:3.0:48.78200000000001,0.0
39996,gontarenkooz,node_8_domain_4_nauss_8_1645876859_4154704.wav,2022-04-30 14:46:14+03:00,2022-04-30,14:46:14,9.589,0.0:0.0:9.589,0.0
39997,harsheva,node_8_domain_1_nauss_8_1645876859_4021438.wav,2022-04-28 10:59:51+03:00,2022-04-28,10:59:51,17.878,0.0:0.0:17.878,0.0
39998,kulikova_eu_ksm,node_7_domain_4_nauss_7_1645875244_3672169.wav,2022-04-29 12:09:48+03:00,2022-04-29,12:09:48,72.798,0.0:1.0:12.798000000000002,0.0


In [30]:
df_3['Duration_sec'] = pd.to_datetime(df_2['Duration_sec'], dayfirst=True)


OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-01-01 00:01:38 present at position 0

In [ ]:
i = 0
for i in range (len(df_3)):
    df_3.Raznica[i] = df_2.time+df_2.Duration_sec

/home/victor/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimelike.py:1189: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  warnings.warn(


TypeError: unsupported operand type(s) for +: 'Timestamp' and 'str'

In [ ]:

# Визуализация полученного дата-фрейма 
"""
fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_1.columns),
                fill_color='paleturquoise',
                align='right'),
    cells=dict(values=df_1,
               fill_color='lavender',
               align='right'),)
])
fig.update_layout(width=1000, height=500)
fig.show()

In [ ]:
# Отсортируем по агентам
df_sort = df_2.sort_values(by=['ccAgentID','datetime'])
df_sort

,ccAgentID,filename,datetime,date,time,Duration
13465,Ekaterina_OP,node_7_domain_2_nauss_7_1645875244_3861667.wav,2022-05-05 10:00:55+03:00,2022-05-05,10:00:55,56.718
18569,Ekaterina_OP,node_8_domain_3_nauss_8_1645876859_4305411.wav,2022-05-05 10:08:46+03:00,2022-05-05,10:08:46,12.187
32614,Ekaterina_OP,node_8_domain_4_nauss_8_1645876859_4305488.wav,2022-05-05 10:09:49+03:00,2022-05-05,10:09:49,42.113
19079,Ekaterina_OP,node_7_domain_2_nauss_7_1645875244_3862681.wav,2022-05-05 10:13:34+03:00,2022-05-05,10:13:34,19.059
16696,Ekaterina_OP,node_8_domain_1_nauss_8_1645876859_4305851.wav,2022-05-05 10:14:18+03:00,2022-05-05,10:14:18,77.767
...,...,...,...,...,...,...
38341,vlasovaoza,node_7_domain_3_nauss_7_1645875244_4294020.wav,2022-05-15 19:55:03+03:00,2022-05-15,19:55:03,130.268
16015,vlasovaoza,node_8_domain_1_nauss_8_1645876859_4768973.wav,2022-05-15 19:59:00+03:00,2022-05-15,19:59:00,46.782
5215,vlasovaoza,node_7_domain_1_nauss_7_1645875244_4294332.wav,2022-05-15 20:10:04+03:00,2022-05-15,20:10:04,46.593
24144,vlasovaoza,node_7_domain_5_nauss_7_1645875244_4294418.wav,2022-05-15 20:15:03+03:00,2022-05-15,20:15:03,81.665


In [ ]:
df_sort['time'].iloc[428817067] - df_sort['time'].iloc[17067]

In [ ]:

numbers = df_sort['time']
i=0
for value in numbers:
    i = i[1]-i[0]
print("Сумма равна", i)



In [ ]:
m=df_1.time
x=df_1.datetime[0+1]

In [ ]:
sum([(x-m) for X in df_1.datetime])
#df_agent=df_1[['ccAgentID'],['datetime']]

In [ ]:
# Itertuples
total = []
for row in df_1.itertuples():
    total.append(row[1] - row[0])

In [ ]:

df_1['time_between_calls']=[]


In [ ]:

def summary(x):
    result = {
        'time_between_calls': x['time'] - x['time']
    }
    return pd.Series(result).round(0)

In [ ]:
df_1.groupby(['ccAgentID']).apply(summary)

In [ ]:
df_1.info()

In [ ]:
df_1.to_csv('gran_for_analize.csv')
df_1.to_pickle('gran_for_analize.pkl')

In [ ]:
print('Кол-во записей\n',len(df_1))

print('Кол-во уникальных агентов\n', len(df_1['ccAgentID'].unique()))

print('Кол-во дней с записями\n', len(df_1['date'].unique()))
print('Уникальные даты\n', (df_1['date'].unique()))


In [ ]:
# Округляем до часов. С помощью функции df_1['hour'] = df_1['datetime'].dt.hour получаем данные в нужном формате INT

df_1['hour'] = df_1['datetime'].dt.hour
print("Округление hours function:/n", df_1['hour'])
df_1.info()

# Округляем datetime до минут - просто для тренировки
df_1['datetime'] = df_1['datetime'].dt.round('min')
print("Округление минуты:/n", df_1['datetime'])



In [ ]:
df_1['ccAgentID'].value_counts().plot(kind='bar', label='calls per agent')
plt.legend()
plt.title('Распределение кол-ва звонков по агентам')
plt.show()


In [ ]:
df.datetime = pd.to_numeric(df.datetime, errors='coerce').fillna(0).astype(np.int64)
df.time = pd.to_numeric(df.time, errors='coerce').fillna(0).astype(np.int64)
df_1.info()
df_1['type'].value_counts().plot(kind='bar', label='time', figsize = (15,6), title = 'Количество calls')
plt.legend()
plt.ylabel('Количество calls')
# plt.show()

In [ ]:
#Построение графика "Распределение звонков по часам"

df_1['hour'].hist(bins=50, figsize = (15,6))
plt.title('Распределение звонков по времени(часам)')
plt.ylabel('Количество звонков')
plt.xlabel('Время звонка (час)')
# plt.show()

In [ ]:
# Распределение звонков во времени
import plotly.express as px
df_1['hour'].plot(kind='bar', label='hour', figsize = (15,6), title = 'dispercia calls')
plt.legend()
plt.ylabel('dispercia calls in time')
# plt.show()



In [ ]:
Agent_hour_min = df_1.groupby('ccAgentID')['hour'].min()
Agent_hour_max = df_1.groupby('ccAgentID')['hour'].max()
# print("Час начала рабочего дня:",Agent_hour_min)
# print("Час конца рабочего дня:", Agent_hour_max)

duration = Agent_hour_max-Agent_hour_min
print("Продолжительность рабочего дня агентов:",duration)

In [ ]:
# Продолжительность рабочего дня datetime

Agent_date_min = df_1.groupby('ccAgentID')['datetime'].min()
Agent_date_max = df_1.groupby('ccAgentID')['datetime'].max()
print("День и час начала рабочего дня/n:",Agent_date_min)
print("День и час конца рабочего дня/n:", Agent_date_max)

duration_date = Agent_date_max-Agent_date_min
print("Продолжительность рабочего дня агентов:",duration_date)

In [ ]:
# saving the dataframe
df_gran = df_1
df_gran.to_csv('file1.csv')
df_gran.to_pickle('/Users/ekaterina/github_repos/FatigueDatasetAnalizeAndCleaning/data-chego-to.pkl')

In [ ]:
# Продолжительность 1 записи

import librosa 
wav_dir = "/Users/ekaterina/github_repos/FatigueApps2023/iFatigue/"
wav_file = "_aLAW_2023-02-28-15-59.aac.wav"
duration = librosa.get_duration(path = wav_dir+wav_file) # в последней версии Librosa вместо filename используется зфер
print("Длительность аудиозаписи", wav_file, "=", duration, "sec")

'''
res = sum(get_duration(filename=str(f)) for f in wav_dir.rglob("*.wav"))
print(res)  
'''